# 안전한 귀향을 위한 안내서

아래 셀들을 전체 실행 하시고 마지막 셀에서 출발지와 도착지를 자유롭게 입력하시면 됩니다.

더욱 자세한 분석과 시각화 과정을 보고 싶으시다면 데이콘 사이트의 **[포스트 코로나, 귀향길이 걱정되는 당신을 위한 <안전한 귀향 안내서>](https://dacon.io/competitions/official/235622/codeshare/1621?page=2&dtype=recent&ptype=pub)** 를 참고해주시기 바랍니다.

## 1. 전처리

### 1.0. 모듈 import

In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import folium
from folium import plugins
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, Image
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import json
import time
warnings.filterwarnings(action='ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
plt.style.use('seaborn')
plt.rc('font', family='Malgun Gothic') 
plt.rc('axes', unicode_minus=False)

In [ ]:
#지도를 시각화하기 위해 다운받아야 할 모듈
!pip install git+https://github.com/python-visualization/branca.git@master

### 1.1 . 발권OD.dat와 지역코드.dat 병합

In [21]:
#발권OD 로드
bo = pd.read_csv("발권 OD.dat", sep='|', header=None)
columns = ["년도", "년월", "운행 일자", "요일 구분", "출발 시도 코드", "출발 시군구 코드", "출발 지역 코드", "도착 시도 코드", "도착 시군구 코드",
           "도착 지역 코드", "철도 통행 인원", "시외버스 통행 인원", "고속버스 통행 인원"] 
bo.columns = columns
#지역코드 로드
jc = pd.read_csv("지역코드.dat", sep='|', header=None)
columns = ["지역 구분", "시도 코드", "시군구 코드", "이용 지역 코드", "시도 명", "시군구 명", "읍면동 명"] 
jc.columns = columns
#지역코드에서 지역명 추출
sido = {}
for i in list(jc['시도 코드'].unique()):
    sido[i] = jc.loc[jc['시도 코드'] == i, '시도 명'].iloc[0]
sigungu = {}
for i in list(jc['시군구 코드'].unique()):
    sigungu[i] = jc.loc[jc['시군구 코드'] == i, '시군구 명'].iloc[0]
jiyeok = {}
for i in list(jc['이용 지역 코드'].unique()):
    jiyeok[i] = jc.loc[jc['이용 지역 코드'] == i, '읍면동 명'].iloc[0]
#발권데이터에서 코드에 따른 지역명 변수 추가 위해 함수 생성 
def sido_change(x):
    try: return sido[int(x)]
    except: return np.nan
def sigungu_change(x):
    try: return sigungu[int(x)]
    except: return np.nan
def jiyeok_change(x):
    try: return jiyeok[int(x)]
    except: return np.nan
#발권데이터에 지역명 추가
bo['출발 시도 명'] = bo['출발 시도 코드'].apply(lambda x: sido_change(x))
bo['도착 시도 명'] = bo['도착 시도 코드'].apply(lambda x: sido_change(x))
bo['출발 시군구 명'] = bo['출발 시군구 코드'].apply(lambda x: sigungu_change(x))
bo['도착 시군구 명'] = bo['도착 시군구 코드'].apply(lambda x: sigungu_change(x))
bo['출발 읍면동 명'] = bo['출발 지역 코드'].apply(lambda x: jiyeok_change(x))
bo['도착 읍면동 명'] = bo['도착 지역 코드'].apply(lambda x: jiyeok_change(x))
bo = bo[["년도", "년월", "운행 일자", "요일 구분", "출발 시도 명", "출발 시군구 명","출발 읍면동 명", "도착 시도 명", "도착 시군구 명", "도착 읍면동 명",
         "철도 통행 인원", "시외버스 통행 인원", "고속버스 통행 인원", "출발 시도 코드", "출발 시군구 코드", "출발 지역 코드", "도착 시도 코드", "도착 시군구 코드", "도착 지역 코드"]]

### 1.2. 안내서를 위한 추가 전처리

In [22]:
bo["버스 통행 인원"] = bo["시외버스 통행 인원"] + bo["고속버스 통행 인원"]
#요일 구분 변수 생성
def weekday(x):
    days = {1:"일", 2:"월", 3:"화", 4:"수", 5:"목", 6:"금", 7:"토"}
    return days[x]
bo['요일 구분'] = bo['요일 구분'].apply(lambda x: weekday(x))
#20년도 데이터 추출
bo_2020 = bo[bo['년도'] == 2020]
bo_2020 = bo_2020[bo_2020["운행 일자"]>=20200119]
#운행일자 dtype 변환
def to_datetime(x):
    date = dt.datetime.strptime(str(x), "%Y%m%d").date()
    return date
bo_2020['운행 일자'] = bo_2020['운행 일자'].apply(lambda x: to_datetime(x))
#출발지 서울, 전국 데이터 분할
bo_2020_drop = bo_2020.copy()
bo_2020 = bo_2020[bo_2020['출발 시도 명'] == "서울특별시"]
bo_2020_drop.dropna(subset=['도착 시도 명', '도착 시군구 명'], inplace=True)

## 2. 안전한 귀향을 위한 안내서

### 2.1. 안내서 제작을 위한 함수 생성
함수 목록은 다음과 같습니다.

1) **to_query** : 인자로 전달한 플랫폼의 출발지역코드를 가진 행을 필터링하기 위하여 쿼리를 작성해주는 함수

2) **covid_count_crawler**: 당일 서울 확진자수와 전국 확진자 수를 반환해주는 함수
  - 크롤링 코드 실행을 위해 [이 곳](https://sites.google.com/a/chromium.org/chromedriver/downloads) 에서 버전에 맞는 **chromedriver.exe를 설치**해주시고 경로를 올바르게 설정해주시길 바랍니다.

3) **find_res_plat** : 도착지로 가는 플랫폼과 출발지에서 가까운 플랫폼을 인자로 받아, 도착지까지 운행하면서 출발지에서 가장 가까운 플랫폼을 출력해주는 함수

4) **where** : 도착지로 도시명이 입력되었을때, 해당 시도 명을 반환해주는 함수

5) **view_map** : 최종 플랫폼으로 선정된 데이터와 출발지(구)를 folium 지도상으로 시각화하는 함수

6) **find_platform** : 출발지, 도착지를 인자로 전달하며, 도착지까지 운행하면서 출발지에서 가장 가까운 플랫폼(철도, 고속버스, 시외버스 각각 1개)을 선정하며, 이 플랫폼들에 해당되는 출발지역코드를 가진 행들만을 필터링 해주는 함수

7) **trans_graph** : 도착지까지 가는 각 플랫폼(철도, 고속버스, 시외버스)의 요일별 일평균 통행량을 그래프로 시각화하는 함수

8) **dest_graph** : 도착지를 기준으로 세부 시군구에 따른 일평균 통행 수단별 통행 인원을 그래프로 시각화하는 함수

9) **pie_plot** : 출발지와 도착지를 모두 고려한 철도와 버스터미널의 통행인원비율을 그래프로 시각화하는 함수

10) **final_guide** : 위 함수들을 모두 종합하여 안내서를 반환하는 함수


In [33]:
# 1) to_query
def to_query(location, loc):
  start = "출발지역코드 == "
  col = "'"
  loc_code = location[loc]
  return (start + col + loc_code + col), loc_code
# 2) covid_count_crawler #chromedriver 경로 확인해주세요!
def covid_count_crawler():
    driver = webdriver.Chrome(executable_path="C:\chromedriver_win32\chromedriver.exe")
    driver.get("https://www.seoul.go.kr/coronaV/coronaStatus.do")
    try:
        covid_seoul = driver.find_element_by_class_name("status-seoul")
        count_seoul = covid_seoul.find_element_by_class_name("counter")
        confirmed_seoul = count_seoul.text        
        covid_korea = driver.find_element_by_class_name("status-korea")
        count_korea = covid_korea.find_element_by_class_name("counter")
        confirmed_korea = count_korea.text
        time = covid_seoul.text[4:22]        
    except: print("Error")
    driver.close()
    return confirmed_seoul, confirmed_korea, time
# 3) find_res_plat
def find_res_plat(dest, near):
  res = ""
  for i in range(len(near)):
    if near[i] in dest:
      res += near[i]
      break
  return res
# 4) where
def where(destination):
    areas = ["부산광역시", "대구광역시", "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", "인천광역시",
            "경기도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "강원도"]
    for a in areas:
        if destination in a:
            destin = a
            return destin
    destin = bo_2020_drop[bo_2020_drop["도착 시군구 명"].str.contains(destination)].iloc[0,7]
    return destin
# 5) view_map
def view_map(start_region, res_plat):
  gps_plat = pd.read_csv("서울시_주요_기차역+터미널_좌표.csv")
  gps_gu = pd.read_csv("서울시 행정구역 시군구 정보 (좌표계_ WGS1984).csv", encoding='cp949')
  geo_path = '02. skorea_municipalities_geo_simple.json'
  geo_str = json.load(open(geo_path, encoding='utf-8'))
  f = folium.Figure(width=700, height=400)
  map = folium.Map(location=[37.5502, 126.982], zoom_start=11).add_to(f)
  map.choropleth(geo_data=geo_str, fill_color = '#BEEFFF')
  gu_xy = gps_gu[gps_gu['시군구명_한글'] == start_region][['위도', '경도']].values
  gu_x = gu_xy[0][0]
  gu_y = gu_xy[0][1]
  folium.Marker([gu_x, gu_y], popup=start_region, tooltip=start_region, icon=folium.Icon(color='red', icon='star')).add_to(map)
  if True:
    for i in range(len(res_plat)):
        plat_xy = gps_plat[gps_plat['이름'] == res_plat[i]][['위도', '경도']].values
        try:
            plat_x = plat_xy[0][0]
            plat_y = plat_xy[0][1]
            folium.Marker([plat_x, plat_y], tooltip=res_plat[i], popup=res_plat[i]).add_to(map)
        except: pass
  else:
     for i in range(len(res_plat)):
        plat_xy = gps_plat[gps_plat['이름'] == res_plat[i]][['위도', '경도']].values
        plat_x = plat_xy[0][0]
        plat_y = plat_xy[0][1]
        folium.Marker([plat_x, plat_y], tooltip=res_plat[i], popup=res_plat[i]).add_to(map)
  display(map)
  return map
# 6) find_platform
def find_platform(df, start_region, destination):
  destins = where(destination)
  df.rename(columns={"출발 지역 코드":"출발지역코드"}, inplace=True)
  near = pd.read_csv("인접플랫폼.csv")
  dest = pd.read_csv("도착터미널구분.csv")
  location = {'서울역' : '1117010700', '용산역' : '1117012800', '수서역' : '1168011500', '청량리역' : '1123010400', '영등포역' : '1156010100', '상봉역' : '1126010200',
            '서울고속버스터미널' : '1165010700', '서울남부터미널' : '1165010800', '동서울종합터미널' : '1121510300', '센트럴시티터미널' : '1165010700'}  
  print("다음으로, 출발지 <" + start_region + ">에서 도착지 <" + destination + ">까지 가는 안전한 귀향길을 안내해 드리겠습니다.")  
  dest_list = dest[dest['도착지'].str.contains(destins)]
  try: dest_express = dest_list['고속버스'].values[0].split(",")
  except: dest_express = []
  try: dest_inter = dest_list['시외버스'].values[0].split(",")
  except: dest_inter = []
  try: dest_train = dest_list['기차'].values[0].split(",")
  except: dest_train = []
  dest_ava_list = []
  if len(dest_express) >= 1: dest_ava_list.append("고속버스")
  if len(dest_inter) >= 1: dest_ava_list.append("시외버스")
  if len(dest_train) >= 1: dest_ava_list.append("기차")
  near_list = near[near['이름'] == start_region]
  near_terminal = near_list['터미널'].values[0].split(",")
  near_train = near_list['기차역'].values[0].split(",")
  res_plat = []    
  temp_plat = find_res_plat(dest_train, near_train)
  if temp_plat == "": pass
  else: res_plat.append(temp_plat)
  temp_plat = find_res_plat(dest_express, near_terminal)
  if temp_plat == "": pass
  else: res_plat.append(temp_plat)
  temp_plat = find_res_plat(dest_inter, near_terminal)
  if temp_plat == "": pass
  else: res_plat.append(temp_plat)   
  print("목적지인 "+ destination + "까지 운행하면서, 출발지와 가장 가까운 플랫폼은 다음과 같습니다.\n")
  if len(res_plat) == 3:
    print(" 1. 기차:", res_plat[0], "\n", "2. 고속버스:", res_plat[1], "\n", "3. 시외버스:", res_plat[2],"\n"*2)
  else:
      if destins == "대구광역시": #데이터 상으로 운행하는 통행 수단이 기차(철도) 하나뿐인 대구광역시 예외처리
          print(" 1. 기차:", res_plat[0],"\n"*2)
      elif destins == "인천광역시": #데이터 상으로 운행하는 통행 수단이 시외버스 하나뿐인 인천광역시 예외처리
          print(" 3. 시외버스:", res_plat[0],"\n"*2)
  query = ""
  count = 0
  loc_code = []
  for i in range(len(res_plat)):
    if res_plat[i] != '':
      q, code = to_query(location, res_plat[i])
      query += q
      count += 1
      loc_code.append(code)
      if (i != len(res_plat)-1):
        query += " or "
  if query[-3:] == 'or ':
    query = query[:-4]
  print("한 눈에 보시기 편하게 플랫폼 별 위치를 지도로 보여드리겠습니다.", "\n")
  map = view_map(start_region, res_plat)  
  return df.query(query), map, loc_code, res_plat
# 7) trans_graph
def trans_graph(df, loc_code, res_plat, destins):
    nums = ["철도 통행 인원", "고속버스 통행 인원", "시외버스 통행 인원"]
    names = ["(기차)", "(고속버스)", "(시외버스)"]
    days = ["월", "화", "수", "목", "금", "토", "일"]
    colors = ["#009770", "#FFDF65", "#E91E63"]
    if destins == "대구광역시":#데이터 상으로 운행하는 통행 수단이 기차(철도) 하나뿐인 대구광역시 예외처리
        df_1 = df.groupby('출발지역코드').get_group(loc_code[0])
        df_1 = df_1[df_1[nums[0]]!=0]
        avgs = []
        for d in range(len(days)):
            new = df_1.groupby("요일 구분").get_group(days[d])
            avg = new[nums[0]].sum()//len(new["운행 일자"].unique())
            avgs.append(avg)
        fig = go.Figure(data=[go.Bar(x=days, y=avgs, marker_color="#009770")])
        fig.update_layout(width=700, template='plotly_dark',title_text="2020년 "+res_plat[0]+names[0]+ " 요일별 일평균 통행인원", )
        fig.show()
    elif destins == "인천광역시":#데이터 상으로 운행하는 통행 수단이 시외버스 하나뿐인 인천광역시 예외처리
        df_1 = df.groupby('출발지역코드').get_group(loc_code[0])
        df_1 = df_1[df_1[nums[2]]!=0]
        avgs = []
        for d in range(len(days)):
            new = df_1.groupby("요일 구분").get_group(days[d])
            avg = new[nums[2]].sum()//len(new["운행 일자"].unique())
            avgs.append(avg)
        fig = go.Figure(data=[go.Bar(x=days, y=avgs, marker_color="#E91E63")])
        fig.update_layout(width=700, template='plotly_dark',title_text="2020년 "+res_plat[0]+names[2]+ " 요일별 일평균 통행인원", )
        fig.show()
    else:
        fig = make_subplots(rows=1, cols=3, subplot_titles=(res_plat[0]+names[0],res_plat[1]+names[1],res_plat[2]+names[2]))
        for c in range(len(loc_code)):
            df_1 = df.groupby('출발지역코드').get_group(loc_code[c])
            df_1 = df_1[df_1[nums[c]]!=0]
            avgs = []
            for d in range(len(days)):
                new = df_1.groupby("요일 구분").get_group(days[d])
                avg = new[nums[c]].sum()//len(new["운행 일자"].unique())
                avgs.append(avg)
            fig.add_trace(go.Bar(x=days, y=avgs, name=nums[c], marker_color=colors[c] ),row=1, col=c+1)
        fig.update_layout(template='plotly_dark',title_text="2020년 요일별 일평균 통행인원")
        fig.show()
# 8) dest_graph
def dest_graph(destination):
    areas = ["부산광역시", "대구광역시", "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", "인천광역시",
            "경기도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "강원도"]
    sigungus = []
    for s in range(7):
        area = bo_2020_drop.groupby('도착 시도 명').get_group(areas[s])
        sigungus.append(list(area["도착 시군구 명"].unique()))
    sigungus.append(["수원시", "평택시", "광명시", "성남시", "화성시", "안산시"])
    sigungus.append(["청주시", "충주시", "제천시", "진천군", "영동군", "음성군"])
    sigungus.append(["천안시", "아산시", "서산시", "당진시", "공주시", "논산시"])
    sigungus.append(["전주시", "익산시", "군산시", "정읍시", "남원시", "임실군"])
    sigungus.append(["순천시", "목포시", "여수시", "나주시", "광양시", "고흥군"])
    sigungus.append(["경주시", "구미시", "포항시", "김천시", "경산시", "안동시"])
    sigungus.append(["진주시", "창원시", "밀양시", "김해시", "통영시", "거제시"])
    sigungus.append(["원주시", "춘천시", "강릉시", "속초시", "동해시", "홍천군"])
    a = areas.index(where(destination))
    kk_20 = bo_2020_drop.groupby('도착 시도 명').get_group(where(destination))
    all_kk = pd.DataFrame(index=range(0,6), columns = ["년도", "도착 시군구 명", "철도 통행 인원", "시외버스 통행 인원", "고속버스 통행 인원"])
    nums = ["철도 통행 인원", "시외버스 통행 인원", "고속버스 통행 인원"]
    for i in range(len(sigungus[a])):
        all_kk["도착 시군구 명"][i] = sigungus[a][i]
        all_kk["년도"][i] = 2020
        kk_20_1 = kk_20[kk_20['도착 시군구 명'].str.contains(sigungus[a][i])]
        for n in range(len(nums)):
            new_20 = kk_20_1[kk_20_1[nums[n]]!=0]
            avg_20 = new_20[nums[n]].sum()//len(new_20["운행 일자"].unique())    
            all_kk[nums[n]][i] = avg_20
    trace1 = go.Bar(x=all_kk["도착 시군구 명"], y=all_kk["철도 통행 인원"], name='철도', marker_color="#009770")
    trace2 = go.Bar(x=all_kk["도착 시군구 명"], y=all_kk["시외버스 통행 인원"], name='시외버스', marker_color="#E91E63")
    trace3 = go.Bar(x=all_kk["도착 시군구 명"], y=all_kk["고속버스 통행 인원"], name='고속버스', marker_color="#FFDF65")
    data = [trace1, trace2, trace3]
    title = "2020년 "+areas[a]+" 일평균 통행 수단 별 통행 인원 분포"
    layout = go.Layout(template='plotly_dark',width=700, title=title, barmode='stack')
    fig = go.Figure(data=data, layout=layout)
    pyo.iplot(fig)
# 9) pie_plot
def pie_plot(destination, result):
    dict_train = {'서울역':'1117010700', '용산역':'1117012800', '수서역': '1168011500', '영등포역':'1156010100', '청량리역': '1123010400'}
    dict_bus = {'서울고속버스터미널':'1165010700', '센트럴시티터미널':'1165010700','동서울종합터미널':'1121510300', '서울남부터미널': '1165010800'}
    result_train = bo[(bo['출발 지역 코드'] == dict_train[result[0]]) & (bo['도착 시도 명'] == destination)]
    result_bus1 = bo[(bo['출발 지역 코드'] == dict_bus[result[1]]) & (bo['도착 시도 명'] == destination)]
    result_bus2 = bo[(bo['출발 지역 코드'] == dict_bus[result[2]]) & (bo['도착 시도 명'] == destination)]
    means = [result_train['철도 통행 인원'].mean(), result_bus1['버스 통행 인원'].mean(), result_bus2['버스 통행 인원'].mean()]
    if dict_bus[result[1]] != dict_bus[result[2]]: 
        ratio = [means[0], means[1], means[2]]
        labels = [result[0], result[1], result[2]]
        df = px.data.tips()
        fig = px.pie(df, values=ratio, names=labels, color_discrete_sequence=["#009770", "#FFDF65", "#E91E63"])
        fig.update_layout(width=600, height=500, template='plotly_dark')
        fig.update_layout(title={'text':'출발지와 목적지 모두 동일한 경우의 철도/버스 평균 인원 비율', 'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'})
        fig.show()
    else:
        ratio = [means[0], means[1]]
        labels = [result[0], result[1]]
        df = px.data.tips()
        fig = px.pie(df, values=ratio, names=labels, color_discrete_sequence=["#009770", "#E91E63"])
        fig.update_layout(width=600, height=500, template='plotly_dark')
        fig.update_layout(title={'text':'출발지와 목적지 모두 동일한 경우의 철도/버스 평균 인원 비율', 'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'})
        fig.show()
# 10) final_guide
def final_guide(start_region, destination):
    new_destin = where(destination)
    print("\n"*2," "*9, "---"*35,"\n")
    print(" "*42, "<< 당신의 안전한 귀향을 위한 안내서 >> \n")
    print(" "*10, "---"*35,"\n"*3)
    print("안녕하세요. 당신의 안전한 귀향을 위한 안내서입니다.\n")
    print("저희 안내서는 사용자가 출발지부터 도착지까지 가는 다양한 경로 중, \n타인과의 접촉을 최소화할 수 있는 '안전한 경로'를 선택하는 데 도움을 줄 수 있습니다.\n\n")
    print("먼저, 오늘의 코로나 확진자 현황을 살펴보겠습니다.\n")
    cf_seoul, cf_korea, time = covid_count_crawler()
    print(time, "코로나 확진자는 다음과 같습니다.\n")
    print("    1. 전국 확진자 : ", cf_korea, "명")
    print("    2. 서울 확진자 : ", cf_seoul, "명\n")
    print("이용에 참고하시길 바라겠습니다. :) \n\n")
    df, map, loc_code, res_plat = find_platform(bo_2020, start_region, destination)
    print("\n출발지는 빨간 표식, 플랫폼은 파란 표식입니다. 플랫폼의 대략적인 위치를 파악하십시오.\n지도는 확대, 축소가 자유롭게 가능하며, 마우스를 마커에 올리면 플랫폼명도 확인하실 수 있습니다.")
    print("\n"*3+ "다음으로 보여드릴 세 그래프는 선정된 플랫폼들의 혼잡도 정보를 제공해드립니다.\n이때 혼잡도는 각 플랫폼의 통행량 정도로 정의하였습니다.")
    print("하단 그래프들이 귀향을 위한 통행 수단 선택에 도움이 되길 바랍니다.\n")
    print("\n첫번째 그래프는 위에서 선정된 기차역/고속버스터미널/시외버스터미널 플랫폼 각각의 혼잡도를 요일별로 비교합니다.\n요일별 평균으로 세분화해서 나타내었으니 참고하시길 바랍니다.\n")
    print("마우스를 그래프에 가져다 대시면 보다 세부적인 정보를 확인하실 수 있습니다.")
    trans_graph(df, loc_code, res_plat, new_destin)
    if ("서울남부터미널" not in res_plat) and (new_destin != "인천광역시") and (new_destin != "대구광역시"):
        print("\n\n\n두 번째 그래프는 출발지가 <", start_region, "> 이고 도착지가 <", destination, "> 인 철도와 버스터미널의 통행인원비율입니다.\n")
        print("이 그래프는 입력받은 출발지와 도착지가 모두 동일한 경우의 통행 수단 별 인원 비율을 제공합니다. \n이 차트에서 자신과 동일한 출발/도착지를 가진 다른 사람들은 어떤 통행 수단을 선택했는가에 대한 비율을 살펴볼 수 있습니다.\n")
        print("마우스를 그래프에 가져다 대시면 보다 세부적인 정보를 확인하실 수 있습니다.")
        pie_plot(new_destin, res_plat)
    print("\n\n\n마지막으로 도착지인 <", destination, "> 을 기준으로, 도착지 인접지역의 통행 수단 별 통행 인원 분포를 보여드리겠습니다.\n") 
    print("이는 추천드린 플랫폼들 뿐만 아니라 전국을 출발지로 두고 도착지가 <", destination, ">와 인접 지역인 통행량 및 통행수단 별 비율을 나타냅니다. \n이를 통해, 고향의 인접지역까지 이동하는 사람들은 얼마나 있는지, 또 통행수단별 비율은 어느 정도인지 확인하실 수 있습니다.\n")
    print("마우스를 그래프에 가져다 대시면 보다 세부적인 정보를 확인하실 수 있습니다.")
    dest_graph(destination)
    print("\n\n저희가 제공하는 안내서는 여기까지입니다.\n안내서가 제공하는 그래프들을 통해 안전한 통행 수단을 선택할 수 있길 바랍니다 :)")
    print("\n\n\n")
    print("\n"*2," "*9, "---"*35,"\n")
    print(" "*24,"<< 저희의 안내서를 통해 목적지인", destination, "까지 안전한 귀향길 되시길 바랍니다. >> \n")
    print(" "*10, "---"*35,"\n"*3)

### 2.2. 안전한 귀향을 위한 안내서
아래 셀을 실행하여 확인하고 싶은 출발지와 도착지를 입력하시면 됩니다.

**[유의할 점]**
1. 코로나 확진자 현황을 크롤링하기 때문에 로딩하는데에 약간의 시간이 소요될 수 있는 점 양해 부탁드립니다.
2. 고속버스터미널과 시외버스터미널이 **동서울종합터미널**로 동일한 경우, 출발지가 같기 때문에 원형 차트에서 두 통행 수단의 인원이 합산되어 나타납니다.
3. 시외버스터미널이 **서울남부터미널**인 경우, 데이터상에서 도착지가 나타나있지 않아 원형 차트가 출력되지 않습니다.
4. **대구광역시**의 경우 기차노선만 존재하여 요일별 차트가 기차역 1개만 출력되며, 원형차트가 출력되지 않습니다.
5. **인천광역시**의 경우 시외버스 노선만 존재하여 요일별 차트가 시외버스터미널 1개만 출력되며, 원형차트가 출력되지 않습니다.
6. **출발지**는 **서울 시내 25개 지역구명**으로 입력하시고, **목적지**는 **시도 단위**로 **아래 예시와 같이 입력**해주시기 바랍니다.

In [38]:
start= input("출발지를 지역구 단위로 입력해주세요.(예시: 종로구) : ")
end = input("목적지를 시 단위로 입력해주세요. (예시: 속초) : ")
final_guide(start, end)

출발지를 지역구 단위로 입력해주세요.(예시: 종로구) : 노원구
목적지를 시 단위로 입력해주세요. (예시: 속초) : 천안


           --------------------------------------------------------------------------------------------------------- 

                                           << 당신의 안전한 귀향을 위한 안내서 >> 

           --------------------------------------------------------------------------------------------------------- 



안녕하세요. 당신의 안전한 귀향을 위한 안내서입니다.

저희 안내서는 사용자가 출발지부터 도착지까지 가는 다양한 경로 중, 
타인과의 접촉을 최소화할 수 있는 '안전한 경로'를 선택하는 데 도움을 줄 수 있습니다.


먼저, 오늘의 코로나 확진자 현황을 살펴보겠습니다.

('20.09.04.00시 기준) 코로나 확진자는 다음과 같습니다.

    1. 전국 확진자 :  198 명
    2. 서울 확진자 :  68 명

이용에 참고하시길 바라겠습니다. :) 


다음으로, 출발지 <노원구>에서 도착지 <천안>까지 가는 안전한 귀향길을 안내해 드리겠습니다.
목적지인 천안까지 운행하면서, 출발지와 가장 가까운 플랫폼은 다음과 같습니다.

 1. 기차: 서울역 
 2. 고속버스: 서울고속버스터미널 
 3. 시외버스: 동서울종합터미널 


한 눈에 보시기 편하게 플랫폼 별 위치를 지도로 보여드리겠습니다. 




출발지는 빨간 표식, 플랫폼은 파란 표식입니다. 플랫폼의 대략적인 위치를 파악하십시오.
지도는 확대, 축소가 자유롭게 가능하며, 마우스를 마커에 올리면 플랫폼명도 확인하실 수 있습니다.



다음으로 보여드릴 세 그래프는 선정된 플랫폼들의 혼잡도 정보를 제공해드립니다.
이때 혼잡도는 각 플랫폼의 통행량 정도로 정의하였습니다.
하단 그래프들이 귀향을 위한 통행 수단 선택에 도움이 되길 바랍니다.


첫번째 그래프는 위에서 선정된 기차역/고속버스터미널/시외버스터미널 플랫폼 각각의 혼잡도를 요일별로 비교합니다.
요일별 평균으로 세분화해서 나타내었으니 참고하시길 바랍니다.

마우스를 그래프에 가져다 대시면 보다 세부적인 정보를 확인하실 수 있습니다.





두 번째 그래프는 출발지가 < 노원구 > 이고 도착지가 < 천안 > 인 철도와 버스터미널의 통행인원비율입니다.

이 그래프는 입력받은 출발지와 도착지가 모두 동일한 경우의 통행 수단 별 인원 비율을 제공합니다. 
이 차트에서 자신과 동일한 출발/도착지를 가진 다른 사람들은 어떤 통행 수단을 선택했는가에 대한 비율을 살펴볼 수 있습니다.

마우스를 그래프에 가져다 대시면 보다 세부적인 정보를 확인하실 수 있습니다.





마지막으로 도착지인 < 천안 > 을 기준으로, 도착지 인접지역의 통행 수단 별 통행 인원 분포를 보여드리겠습니다.

이는 추천드린 플랫폼들 뿐만 아니라 전국을 출발지로 두고 도착지가 < 천안 >와 인접 지역인 통행량 및 통행수단 별 비율을 나타냅니다. 
이를 통해, 고향의 인접지역까지 이동하는 사람들은 얼마나 있는지, 또 통행수단별 비율은 어느 정도인지 확인하실 수 있습니다.

마우스를 그래프에 가져다 대시면 보다 세부적인 정보를 확인하실 수 있습니다.




저희가 제공하는 안내서는 여기까지입니다.
안내서가 제공하는 그래프들을 통해 안전한 통행 수단을 선택할 수 있길 바랍니다 :)






           --------------------------------------------------------------------------------------------------------- 

                         << 저희의 안내서를 통해 목적지인 천안 까지 안전한 귀향길 되시길 바랍니다. >> 

           --------------------------------------------------------------------------------------------------------- 



